In [7]:
import gurobipy as gp
import pandas as pd
import numpy as np
import math
import re
from gurobipy import *

In [8]:
fish = pd.read_excel("fish.xlsx")
species_meta = pd.read_excel("species.xlsx")

In [9]:
species_cols = [c for c in fish.columns if c.startswith("SPECIES")]
for col in species_cols:
    fish[col] = fish[col].fillna(0)
    fish[col] = (fish[col].astype(float) != 0).astype(int)
    
print("Final cleaned dataset shape:", fish.shape)

Final cleaned dataset shape: (89, 32)


In [10]:
clean_species = {}

for col in species_cols:
    species_name = col.strip()
    if species_name not in clean_species:
        clean_species[species_name] = fish[col]
    else:
        # Combine duplicate columns
        clean_species[species_name] = clean_species[species_name].combine(fish[col], func=lambda a,b: max(a,b))

In [11]:
species_df = pd.DataFrame(clean_species)
fish_clean = fish.drop(columns=species_cols).join(species_df)
species_cols_final = list(species_df.columns)
fish_clean["species_sum"] = fish_clean[species_cols_final].sum(axis=1)

fish_clean = fish_clean[fish_clean["species_sum"] > 0].copy()
fish_clean.drop(columns=["species_sum"], inplace=True)
fish_clean.reset_index(drop=True, inplace=True)

fish_clean = (
    fish_clean
    .sort_values(["Coordinates", "Catches"], ascending=[True, False])
    .drop_duplicates(subset="Coordinates", keep="first")
    .reset_index(drop=True)
)

print(fish_clean.shape)
print("Final cleaned dataset shape:", fish_clean.shape)

(68, 32)
Final cleaned dataset shape: (68, 32)


In [12]:
species_cols = [c for c in fish_clean.columns if c.startswith("SPECIES")]

def extract_species_name(colname: str) -> str:
    """Get the text inside parentheses: 'SPECIES (Walleye)' -> 'Walleye'."""
    m = re.search(r'\((.*?)\)', colname)
    if m:
        return m.group(1).strip()
    return colname.replace("SPECIES", "").replace("(", "").replace(")", "").strip()

fish_species_names = sorted({extract_species_name(c) for c in species_cols})

def norm_name(s: str) -> str:
    """Normalize species names for matching: lowercase, letters only."""
    return re.sub(r'[^a-z]', '', str(s).lower())

# normalized names in each source
fish_species_norm = {s: norm_name(s) for s in fish_species_names}
species_meta["Fish_norm"] = species_meta["Fish"].apply(norm_name)

# ---------------------------------------------------
# 3. Map fish species to species_meta rows
# ---------------------------------------------------
species_mapping = {}  # key: fish species name, value: index in species_meta
unmatched_species = []

for s in fish_species_names:
    norm_s = fish_species_norm[s]
    matches = species_meta.index[species_meta["Fish_norm"] == norm_s].tolist()
    if matches:
        species_mapping[s] = matches[0]
    else:
        unmatched_species.append(s)

print("Matched species:", list(species_mapping.keys()))
print("Unmatched species:", unmatched_species)

Matched species: ['American shad', 'Black crappie', 'Carpe', 'Catfish', 'Common mullet', 'Common shiner', 'Creek chub', 'Goldeye', 'Lake trout', 'Largemouth bass', 'Muskellunge', 'Northern pike', 'Pumpkinseed', 'Redhorse', 'Rock bass', 'Sauger', 'Sheephead', 'Smallmouth Bass', 'Striped bass', 'Sturgeon', 'Sunfish', 'Tench', 'Walleye', 'Yellow bullhead', 'Yellow perch']
Unmatched species: []


In [13]:
TOP_N_SITES = 20

fish_clean = (
    fish_clean
    .sort_values("Catches", ascending=False)
    .head(TOP_N_SITES)
    .reset_index(drop=True)
)

print("Reduced dataset shape:", fish_clean.shape)
print("Top sites used (Name and Catches):")
display(fish_clean[["Name", "Catches"]])

Reduced dataset shape: (20, 32)
Top sites used (Name and Catches):


,Name,Catches
0,Lac Saint-François,1477
1,Fleuve st laurent,1303
2,Bassin du Pêcheur,741
3,Canal de Lachine,601
4,Ruisseau Bouchard,552
5,Baie de Quenneville,528
6,Ruisseau Hotte,515
7,Canal Lachine,506
8,Bassin de Chambly,455
9,parc bellerive,416


In [39]:
import math
import re
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# ============================================================
#  PART 1: INTERACTIVE USER INPUT
# ============================================================

def get_input_list(prompt_text):
    raw = input(prompt_text)
    if not raw.strip(): return []
    return [x.strip() for x in raw.split(',')]

print("\n" + "="*60)
print("      >>> FISHING TRIP CONFIGURATOR (EFFICIENCY MODE) <<<")
print("="*60)

# 1. Target Species
print("\n[1] Target Species")
print("    Examples: Smallmouth Bass, Walleye, Northern pike")
user_species = get_input_list("👉 Enter species: ")
if not user_species: user_species = ["Smallmouth Bass", "Walleye"]

# 2. Preferred Bait
print("\n[2] Preferred Baits")
print("    Options: soft, hard, natural")
user_baits_raw = get_input_list("👉 Enter baits: ")
user_baits = [b.lower() if b.lower()!='flies' else 'natural' for b in user_baits_raw]
if not user_baits: user_baits = ["soft"]

# 3. Rod
print("\n[3] Preferred Rod")
print("    Options: short, medium, long")
user_rods = [r.lower() for r in get_input_list("👉 Enter rods: ")]
if not user_rods: user_rods = ["medium"]

# --- Parameters ---
TIME_COST_PER_HOUR = 2.0   
RELEASE_UTILITY    = 0.8  
KEEP_BONUS         = 1

# Bonus Weights
BONUS_SPECIES = 1.7
BONUS_BAIT    = 0.5
BONUS_ROD     = 0.1
BYCATCH_FACTOR = 0.3

# Base Utility Weights
W_HABITAT = 0.2
W_SIZE    = 0.2
W_THREAT  = 0.1
W_STATUS  = 0.1

print(f"\n✅ Configured for Efficiency: Stops when Limit is reached.")
print("="*60 + "\n")


# ============================================================
#  PART 2: OPTIMIZATION ENGINE
# ============================================================

# --- 2.1 Utilities ---
def parse_dms_coord(coord_str):
    if not isinstance(coord_str, str): return np.nan, np.nan
    clean_s = re.sub(r"[^0-9\.NSEWnsew\-]", " ", coord_str).strip().upper()
    matches = re.findall(r'(\d+)\s+(\d+)\s+(\d+(?:\.\d+)?)\s*([NSEW])', clean_s)
    if len(matches) >= 2:
        def dms(d, m, s, h):
            val = float(d) + float(m)/60 + float(s)/3600
            return -val if h in ['S', 'W'] else val
        return dms(*matches[0]), dms(*matches[1])
    d = re.findall(r'(-?\d+\.\d+)', clean_s)
    if len(d) >= 2: return float(d[0]), float(d[1])
    return np.nan, np.nan

def haversine(lat1, lon1, lat2, lon2):
    if any(pd.isna([lat1, lon1, lat2, lon2])): return np.nan
    R, p = 6371.0, math.pi / 180
    a = 0.5 - math.cos((lat2-lat1)*p)/2 + math.cos(lat1*p) * math.cos(lat2*p) * (1-math.cos((lon2-lon1)*p))/2
    return 2 * R * math.asin(math.sqrt(a))

def extract_name(c): 
    m = re.search(r"\((.*?)\)", c)
    return m.group(1).strip() if m else c.replace("SPECIES", "").strip()

def norm_str(s): return re.sub(r"[^a-z]", "", str(s).lower())

# --- 2.2 Data Loading ---
fish_clean = fish_clean.sort_values("Catches", ascending=False).drop_duplicates(subset=["Name"], keep="first").sort_index().reset_index(drop=True)
fish_clean["District"] = fish_clean["District"].fillna("Unknown")

fish_species_names = sorted({extract_name(c) for c in species_cols_final})
fish_species_norm = {s: norm_str(s) for s in fish_species_names}
species_meta["Fish_norm"] = species_meta["Fish"].apply(norm_str)
species_mapping = {}
for s in fish_species_names:
    m = species_meta.index[species_meta["Fish_norm"] == fish_species_norm[s]].tolist()
    if m: species_mapping[s] = m[0]

S = sorted(species_mapping.keys())
I = list(fish_clean.index)

species_meta["Lure_Clean"] = species_meta["Lure"].dropna().str.strip().str.lower().replace("flies", "natural")
T = sorted(species_meta["Lure_Clean"].dropna().unique().tolist())
districts = sorted(fish_clean["District"].unique().tolist())

lure_map, w_score_map, threat_map, status_map = {}, {}, {}, {}
for s in S:
    idx = species_mapping[s]
    lure_map[s] = str(species_meta.loc[idx, "Lure_Clean"])
    w_score_map[s] = species_meta.loc[idx, "Weighted Score"]
    st = str(species_meta.loc[idx, "IUCN Status"]).lower()
    status_map[s] = 1.0 if "critically" in st else (0.8 if "endangered" in st else 0.0)
    th = str(species_meta.loc[idx, "Threat to humans"]).lower()
    threat_map[s] = 1.0 if "pest" in th else 0.0

w_min, w_max = min(w_score_map.values()), max(w_score_map.values())
def norm(v,mn,mx): return 0.5 if mx==mn else (v-mn)/(mx-mn)
size_hat = {(i,s): norm(w_score_map[s], w_min, w_max) for i in I for s in S}

s_min, s_max = fish_clean["Star"].min(), fish_clean["Star"].max()
habitat_hat = {i: norm(fish_clean.loc[i,"Star"], s_min, s_max) for i in I}
r_class = {i: "medium" for i in I}

col_map = {extract_name(c): c for c in species_cols_final if extract_name(c) in S}
presence = {(i, s): fish_clean.loc[i, col_map[s]] for i in I for s in S}
Omega = [(i, s, lure_map[s]) for i in I for s in S 
         if presence[(i,s)] == 1 and lure_map[s] in T]

# --- 2.3 Efficiency ---
avg_time_per_fish = {s: 1.5 for s in S} 
hard_list = ["Muskellunge", "Sturgeon", "Walleye", "Lake trout", "Northern pike", "Catfish", "Striped bass", "Carpe", "Goldeye"]
for hs in hard_list: 
    if hs in avg_time_per_fish: avg_time_per_fish[hs] = 4.0

is_easy = {s: (avg_time_per_fish[s] < 3.0) for s in S}
mean_c = fish_clean["Catches"].mean()
if mean_c == 0: mean_c = 1.0
Catch_Rate = {}
for (i, s, t) in Omega:
    base_r = 1.0 / avg_time_per_fish[s]
    site_mult = max(0.5, min(1.5, 0.8 + 0.4*(fish_clean.loc[i,"Catches"]/mean_c)))
    Catch_Rate[(i,s,t)] = base_r * site_mult

# --- 2.4 Travel ---
SPEED_D, SPEED_W, PENALTY = 60.0, 4.0, 0.5
DEPOT = (45.5048, -73.5772)
coords = {i: parse_dms_coord(fish_clean.loc[i, "Coordinates"]) for i in I}
coords["depot"] = DEPOT
I0 = ["depot"] + I

tau_d, tau_w = {}, {}
for i in I0:
    for j in I0:
        if i == j: tau_d[(i,j)] = tau_w[(i,j)] = 0.0
        else:
            d = haversine(*coords[i], *coords[j])
            if pd.isna(d): tau_d[(i,j)] = tau_w[(i,j)] = 1000.0
            else:
                tau_d[(i,j)] = d / SPEED_D
                tau_w[(i,j)] = (d / SPEED_W) if d < 100 else 1000.0

# --- 2.5 Model ---
m = gp.Model("FishingTrip")

g = m.addVars(districts, vtype=GRB.BINARY, name="g")
y = m.addVars(I, vtype=GRB.BINARY, name="y")
x = m.addVars(Omega, vtype=GRB.BINARY, name="x")
h = m.addVars(Omega, lb=0.0, vtype=GRB.CONTINUOUS, name="h")
b = m.addVars(Omega, vtype=GRB.BINARY, name="b") 
K = m.addVars(Omega, vtype=GRB.INTEGER, lb=0, name="K")
Total_Catch_Var = m.addVars(Omega, lb=0.0, vtype=GRB.CONTINUOUS, name="TC")

v = m.addVars(I0, I0, vtype=GRB.BINARY, name="v")
D = m.addVars(I0, I0, vtype=GRB.BINARY, name="Drive")
W = m.addVars(I0, I0, vtype=GRB.BINARY, name="Walk")
U = m.addVars(I, vtype=GRB.INTEGER, lb=0, name="U") 

# Params (Fixed Syntax Error Here)
alpha = {s: (BONUS_SPECIES if s in user_species else 0.0) for s in S}
beta = {t: (BONUS_BAIT if t in user_baits else 0.0) for t in T}
delta = {"short": 0.0, "medium": 0.0, "long": 0.0}
for r in user_rods: 
    if r in delta: 
        delta[r] = BONUS_ROD

# --- OBJECTIVE ---
expr_util = 0
total_time_expr = 0

travel_time_expr = gp.quicksum((tau_d[(i,j)]+PENALTY)*D[i,j] + tau_w[(i,j)]*W[i,j] 
                               for i in I0 for j in I0 if i!=j)
total_time_expr += travel_time_expr

for (i,s,t) in Omega:
    rod_bonus = delta.get(r_class[i], 0.0)
    
    # Weighted Base Utility
    base_util = (W_HABITAT * habitat_hat[i]) + \
                (W_SIZE * size_hat[(i,s)]) + \
                (W_THREAT * threat_map[s]) + \
                (W_STATUS * status_map[s])
    
    # Experience Utility
    expr_util += (base_util + alpha[s] + beta[t] + rod_bonus) * h[(i,s,t)]
    
    # Catch Utility
    expr_util += (RELEASE_UTILITY * Total_Catch_Var[(i,s,t)]) + (KEEP_BONUS * K[(i,s,t)])
    
    total_time_expr += h[(i,s,t)]

m.setObjective(expr_util - (TIME_COST_PER_HOUR * total_time_expr), GRB.MAXIMIZE)

# --- Constraints ---
m.addConstr(gp.quicksum(g[d] for d in districts) == 1)
for i in I: m.addConstr(y[i] <= g[fish_clean.loc[i,"District"]])

H_MAX, MIN_T = 4.0, 0.25

for (i,s,t) in Omega:
    m.addConstr(x[(i,s,t)] <= y[i])
    m.addConstr(h[(i,s,t)] <= H_MAX * x[(i,s,t)])
    m.addConstr(h[(i,s,t)] >= MIN_T * x[(i,s,t)])
    
    target_c = Catch_Rate[(i,s,t)] * h[(i,s,t)]
    bycatch_sum = 0
    if is_easy[s]:
        for (idx_o, s_o, t_o) in Omega:
            if idx_o == i and t_o == t and s_o != s and is_easy[s_o]:
                bycatch_sum += (Catch_Rate[(i,s,t)] * BYCATCH_FACTOR * h[(idx_o, s_o, t_o)])
    m.addConstr(Total_Catch_Var[(i,s,t)] == target_c + bycatch_sum)
    
    m.addConstr(K[(i,s,t)] <= Total_Catch_Var[(i,s,t)])
    m.addConstr(K[(i,s,t)] <= 10.0 * y[i])

for i in I:
    site_h = gp.quicksum(h[(idx, s, t)] for (idx, s, t) in Omega if idx == i)
    m.addConstr(site_h <= H_MAX * y[i])

m.addConstr(gp.quicksum(v["depot",j] for j in I) == 1)
m.addConstr(gp.quicksum(v[i,"depot"] for i in I) == 1)
for i in I:
    m.addConstr(gp.quicksum(v[i,j] for j in I0 if j!=i) == y[i])
    m.addConstr(gp.quicksum(v[j,i] for j in I0 if j!=i) == y[i])
for i in I0:
    for j in I0: 
        if i!=j: m.addConstr(D[i,j] + W[i,j] == v[i,j])

N = len(I)
for i in I:
    for j in I: 
        if i!=j: m.addConstr(U[i] - U[j] + N*v[i,j] <= N-1)

T_CAP = 12.0
m.addConstr(total_time_expr <= T_CAP)
for (i,j), val in tau_d.items():
    if i!=j and val*SPEED_D > 3.0: m.addConstr(W[i,j] == 0)

Limit_s = {s: 6 for s in S}
for s in S: m.addConstr(gp.quicksum(K[(i,s,t)] for i in I for t in T if (i,s,t) in Omega) <= Limit_s[s])
GLOBAL_LIMIT = 6
m.addConstr(gp.quicksum(K[(i,s,t)] for (i,s,t) in Omega) <= GLOBAL_LIMIT)

# ============================================================
#  PART 3: OUTPUT
# ============================================================

print("\n--- Optimizing ---")
m.Params.TimeLimit = 60
m.optimize()

if m.status in [GRB.OPTIMAL, GRB.TIME_LIMIT]:
    curr, route_ids = "depot", []
    t_d_tot, t_p_tot, t_w_tot = 0,0,0
    while True:
        found = False
        for j in I0:
            if curr != j and v[curr, j].X > 0.5:
                mode = "DRIVE 🚗" if D[curr, j].X > 0.5 else "WALK 🚶"
                t = tau_d[(curr, j)] if "DRIVE" in mode else tau_w[(curr, j)]
                p = PENALTY if "DRIVE" in mode else 0.0
                if "DRIVE" in mode: t_d_tot+=t; t_p_tot+=p
                else: t_w_tot+=t
                route_ids.append((curr, j, mode, t, p))
                curr = j; found = True; break
        if not found or curr == "depot": break

    fishing_data = []
    tot_fish_time, tot_catch_int, tot_kept_int = 0, 0, 0
    
    for i in I:
        if y[i].X > 0.5:
            name = fish_clean.loc[i, "Name"]
            disp_name = f"{name} (ID:{i})"
            acts = []
            site_time = 0.0
            
            for (idx, s, t) in Omega:
                if idx == i:
                    hr_spent = h[(idx,s,t)].X
                    catch_val = Total_Catch_Var[(idx,s,t)].X
                    kp = int(K[(idx,s,t)].X)
                    
                    c_int = int(catch_val + 1e-5)
                    if c_int < kp: c_int = kp
                    
                    if hr_spent > 0.01 or c_int > 0:
                        type_lbl = "TARGET" if hr_spent > 0.01 else "RANDOM"
                        acts.append((s, t, c_int, kp, type_lbl))
                        site_time += hr_spent
                        tot_catch_int += c_int
                        tot_kept_int += kp
            
            if acts:
                fishing_data.append((disp_name, site_time, acts))
                tot_fish_time += site_time

    tot_rel_int = tot_catch_int - tot_kept_int
    total_duration = t_d_tot + t_p_tot + t_w_tot + tot_fish_time

    print("\n" + "━"*95)
    print(f"🎣  FISHING TRIP SUMMARY (Obj: {m.objVal:.2f})")
    print("━"*95)
    print(f"⏱️  Duration : {total_duration:.2f} / {T_CAP} hrs")
    if total_duration < T_CAP - 0.5:
        print(f"   (⚠️ Trip ended early! Bag limit reached or cost outweighed benefit)")
    
    print(f"🐟  Real Catch: {tot_catch_int}  (Keep: {tot_kept_int} | Release: {tot_rel_int})")
    print(f"📍  Districts : {fish_clean.loc[I[0],'District']}")
    print("-" * 95)
    
    print("\n🚗  TRIP ITINERARY")
    for (u, v_node, mode, t, p) in route_ids:
        nu = "McGill" if u == "depot" else f"{fish_clean.loc[u, 'Name']} ({u})"
        nv = "McGill" if v_node == "depot" else f"{fish_clean.loc[v_node, 'Name']} ({v_node})"
        ts = f"{t:.2f}h" + (f" (+{p:.2f}h prep)" if p>0 else "")
        print(f"  {nu:<25} -> {nv:<25} : {mode:<8} [{ts}]")

    print("\n📊  DETAILED CATCH LOG (Itemized List)")
    if fishing_data:
        for (site_name, st, acts) in fishing_data:
            print(f"\n  📍 {site_name} (Total Fishing Time: {st:.2f}h)")
            print(f"     {'TYPE':<8} {'SPECIES':<20} {'BAIT':<8} {'STATUS'}")
            print(f"     {'-'*8} {'-'*20} {'-'*8} {'-'*15}")
            cnt = 0
            for (s, t, catch, kp, type_lbl) in acts:
                if catch == 0: 
                    print(f"     {type_lbl:<8} {s:<20} {t:<8} (No Catch)")
                for k in range(catch):
                    status = "✅ KEEP" if k < kp else "👋 REL"
                    print(f"     {type_lbl:<8} {s:<20} {t:<8} {status}")
                    cnt += 1
            if cnt == 0 and st > 0: print("     (No fish caught here)")
    else:
        print("  (No fishing activities planned)")
    print("="*95)
else:
    print("No optimal solution found.")


      >>> FISHING TRIP CONFIGURATOR (EFFICIENCY MODE) <<<

[1] Target Species
    Examples: Smallmouth Bass, Walleye, Northern pike


👉 Enter species:  Largemouth bass



[2] Preferred Baits
    Options: soft, hard, natural


👉 Enter baits:  soft



[3] Preferred Rod
    Options: short, medium, long


👉 Enter rods:  medium



✅ Configured for Efficiency: Stops when Limit is reached.


--- Optimizing ---
Set parameter TimeLimit to value 60
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900HX, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads

Non-default parameters:
TimeLimit  60

Optimize a model with 1924 rows, 1871 columns and 6257 nonzeros
Model fingerprint: 0x4f2cf96d
Variable types: 202 continuous, 1669 integer (1548 binary)
Coefficient statistics:
  Matrix range     [2e-01, 2e+01]
  Objective range  [5e-01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective -4.3145495
Presolve removed 1239 rows and 1492 columns
Presolve time: 0.06s
Presolved: 685 rows, 379 columns, 2259 nonzeros
Variable types: 43 continuous, 336 integer (271 binary)

Root relaxation: objective 1.694159e+01, 64 iterations, 0.00 seconds (